<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
# --- Step 1: Install dependencies ---
!apt update -qq
!apt install -y git python3-venv python3-pip wget

# --- Step 2: Clone the Web UI (MY-AI-Gizmo-working) ---
import os
from pathlib import Path

if not Path("MY-AI-Gizmo-working").exists():
    !git clone https://github.com/gitleon8301/MY-AI-Gizmo-working.git
os.chdir("MY-AI-Gizmo-working")

# --- Step 3: Create models folder ---
!mkdir -p user_data/models

# --- Step 4: Download CPU-compatible LLaMA 7B 4-bit GGUF model ---
model_repo = "TheBloke/LLaMA-7b-GGUF"  # Hugging Face repo name
!python download-model.py {model_repo}

# --- Step 5: Set CPU-safe flags ---
command_line_flags = "--cpu --load-in-8bit"  # CPU-compatible
api = True  # optional, enables API

# Add API flags if requested
if api:
    for param in ['--api', '--public-api']:
        if param not in command_line_flags:
            command_line_flags += f" {param}"

# --- Step 6: Find downloaded model automatically ---
import glob

# leta rekursivt i undermappar
model_files = glob.glob("user_data/models/**/*.gguf", recursive=True)
if len(model_files) == 0:
    raise ValueError("No .gguf model found in user_data/models/")
model_path = model_files[0]
print("Using model:", model_path)


# --- Step 7: Launch the Web UI on CPU ---
cmd = f"./start_linux.sh {command_line_flags} --share --model {model_path}"
print("Running command:", cmd)
!$cmd
